In [517]:
import numpy as np
import pandas as pd
import nltk
import sys

In [518]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Phili\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [519]:
df = pd.read_csv("data.csv", encoding='utf-8', index_col=0)
df

,Embedded_text,majority_sentiment
Unnamed: 0,,
0,Unglaublich. Ich hatte @JanJosefLiefers noch...,2
1,Kicking of our @EPPGroup bureau meeting in R...,3
2,Die Schulbürokratie hält mit der #Corona-Herau...,2
4,"""Diese Regierung hat das Vertrauen der Bevölke...",2
5,Unsere Migrationspolitik muss ethischen Ansprü...,2
...,...,...
1763,Am Freitag stellen wir den Entwurf unseres Wah...,3
1767,Gestern habe ich für den Endspurt des Volksbeg...,3
1772,Check: \n Wir verlängern den vereinfachten Zug...,1


In [520]:
def preProcess(tweet):
    tokens = nltk.word_tokenize(tweet)
    tokens = [w.lower() for w in tokens]
    tokens = [w for w in tokens if w.isalpha()]
    return tokens

In [521]:
def tweetToFeatures(tokensList):
    fdist = nltk.FreqDist(tokensList)
    dictionary = dict(fdist)
    s = pd.Series(dictionary)
    return s

In [522]:
tokens = preProcess(df["Embedded_text"][0])


In [523]:
def filter(tokens):
    return [w for w in tokens if w.isalpha()]

In [524]:
tokens = filter(tokens)

In [525]:
features = tweetToFeatures(tokens)

In [526]:
bagOfWordsPerTweet = []
for tweet in df["Embedded_text"]:
    tokens = preProcess(tweet)
    bagOfWords = tweetToFeatures(tokens)
    bagOfWordsPerTweet.append(bagOfWords)

C:\Users\Phili\AppData\Local\Temp/ipykernel_22784/1407671057.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(dictionary)


In [527]:
bagOfWords = pd.DataFrame(bagOfWordsPerTweet)

In [528]:
bagOfWords = bagOfWords.fillna(0)

In [529]:
def filterUniqueWords(featureSpace):
        toDelete = []
        newFs = featureSpace.copy()
        for col in newFs:
            if(sum(newFs[col]) == 1.0):
                toDelete.append(col)
        for word,value in newFs.astype(bool).sum(axis=0).iteritems():
            if(value == 1):
                toDelete.append(word)
        newFs = newFs.drop(list(set(toDelete)), axis=1)
        return newFs

In [530]:
bagOfWords = filterUniqueWords(bagOfWords)

In [531]:
vectors = bagOfWords.values
vectors

array([[1., 2., 2., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [532]:
from sklearn.model_selection import train_test_split

In [533]:
x_train, x_test, y_train, y_test = train_test_split(vectors, df['majority_sentiment'], test_size=0.2, random_state = 1)

<h1>Naive Bayes<h1>

In [534]:
#clf = MultinomialNB().fit(x_train, y_train)

In [535]:
from sklearn.naive_bayes import MultinomialNB

In [536]:
clf = MultinomialNB().fit(x_train, y_train)

In [537]:
from sklearn.metrics import make_scorer, accuracy_score, balanced_accuracy_score, f1_score, matthews_corrcoef, precision_score, recall_score

In [538]:
scoring = [make_scorer(accuracy_score),
           make_scorer(balanced_accuracy_score),
           make_scorer(f1_score, average='macro'),
           make_scorer(f1_score, average='micro'),
           make_scorer(f1_score, average=None, labels = [1]),
           make_scorer(f1_score, average=None, labels = [2]),
           make_scorer(f1_score, average=None, labels = [3]),
           make_scorer(f1_score, average='weighted'),
           make_scorer(matthews_corrcoef),
           make_scorer(precision_score, average=None, labels = [1]),
           make_scorer(precision_score, average=None, labels = [2]),
           make_scorer(precision_score, average=None, labels = [3]),
           make_scorer(precision_score, average='macro'),
           make_scorer(precision_score, average='micro'),
           make_scorer(precision_score, average='weighted'),
           make_scorer(recall_score, average=None, labels = [1]),
           make_scorer(recall_score, average=None, labels = [2]),
           make_scorer(recall_score, average=None, labels = [3]),
           make_scorer(recall_score, average='macro'),
           make_scorer(recall_score, average='micro'),
           make_scorer(recall_score, average='weighted')
]

In [539]:
from sklearn.model_selection import StratifiedShuffleSplit

In [540]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

In [541]:
from sklearn.model_selection import cross_val_score

In [542]:
def report(scoring):
    for score in scoring:
        scores = cross_val_score(clf, vectors, df['majority_sentiment'], scoring=score, cv = sss)
        print(scores.mean())

In [543]:
report(scoring)

0.6504201680672269
0.6508727086556754
0.6534978563313735
0.6504201680672269
0.6817800893595021
0.6400899837243321
0.6386234959102861
0.6507890526628825
0.4631809377241069
0.7186783244999893
0.6259243910480322
0.6307152765432777
0.6584393306970997
0.6504201680672269
0.6531796768340614
0.6494845360824743
0.6560747663551402
0.6470588235294118
0.6508727086556754
0.6504201680672269
0.6504201680672269


<h1>SVM<h1>

In [544]:
from sklearn import svm

In [545]:
clf = svm.SVC()

In [546]:
clf.fit(x_train, y_train)

SVC()

In [547]:
report(scoring)

0.5759103641456582
0.5390560502069601
0.545265299470431
0.5759103641456582
0.4378824237389692
0.5666187015353701
0.6312947731369538
0.5593582533807242
0.33832725566554045
0.7202397784847413
0.6162215774768967
0.5320629610651949
0.6228414390089443
0.5759103641456582
0.6084162474678505
0.31546391752577313
0.525233644859813
0.7764705882352941
0.5390560502069601
0.5759103641456582
0.5759103641456582


<h1>Pipeline for comparing methods<h1>

In [548]:
from sklearn.feature_extraction.text import CountVectorizer

In [549]:
from sklearn.preprocessing import MinMaxScaler

In [550]:
scaler = MinMaxScaler()

In [551]:
from sklearn.ensemble import RandomForestClassifier

In [552]:
from sklearn.linear_model import SGDClassifier

In [553]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [554]:
import spacy

In [555]:
nlp = spacy.load('de_core_news_sm')

In [556]:
from pprint import pprint

In [557]:
def lemmatize(text):
    doc = nlp(text)
    return [word.lemma_ for word in doc]

In [558]:
normalizationMethods = ["None", "lower", "lemmatization", "lemmatizationAndLower"]
def normalize(tweetText, approach):
    if(approach == "None"):
        tokens = nltk.word_tokenize(tweetText)
        return tokens
    elif(approach == "lower"):
        tokens = nltk.word_tokenize(tweetText)
        tokens = [w.lower() for w in tokens]
        return tokens
    elif(approach == "lemmatization"):
        doc = nlp(tweetText)
        tokens = lemmatize(tweetText)
        return tokens
    elif(approach == "lemmatizationAndLower"):
        doc = nlp(tweetText)
        tokens = lemmatize(tweetText)
        tokens = [w.lower() for w in tokens]
        return tokens

In [559]:
filterMethods = ["None", "punctuation", "punctuationAndStopwords"]
def filter(tokens, approach):
    if(approach == "None"):
        return tokens
    elif(approach == "punctuation"):
        return [w for w in tokens if w.isalpha()]
    elif(approach == "punctuationAndStopwords"):
        return [w for w in tokens if w.isalpha() and w.lower()]

In [560]:
featureMethods = ["tf"]
def textToFeatures(tokensList, approach):
    if(approach == "tf"):
        fdist = nltk.FreqDist(tokensList)
        dictionary = dict(fdist)
        s = pd.Series(dictionary)
        return s

        

In [561]:
featureSpaceFilterMethods = ["None", "uniqueWords"]
def filterFs(featureSpace, filterMethod):
    if(filterMethod == "None"):
        return featureSpace
    elif(filterMethod == "uniqueWords"):
        toDelete = []
        newFs = featureSpace.copy()
        for col in newFs:
            if(sum(newFs[col]) == 1.0):
                toDelete.append(col)
        for word,value in newFs.astype(bool).sum(axis=0).iteritems():
            if(value == 1):
                toDelete.append(word)
        newFs = newFs.drop(list(set(toDelete)), axis=1)
        return newFs

In [562]:
featureSpaceTransformationMethods = ["None"]
def transformFeatureSpace(featureSpace, transformationMethod):
    if(transformationMethod == "None"):
        return featureSpace
        

In [563]:
def buildFeatureSpace(normalizationApproach, filterApproach, featureMethod, filterFeatureSpace):
    featuresPerTweet = []
    for tweet in df["Embedded_text"]:
        tokens = normalize(tweet, normalizationApproach)
        tokens = filter(tokens, filterApproach)
        features = textToFeatures(tokens, featureMethod)
        featuresPerTweet.append(features)
    featureSpace = pd.DataFrame(featuresPerTweet)
    featureSpace = featureSpace.fillna(0)
    featureSpace = filterFs(featureSpace, filterFeatureSpace)
    return featureSpace

In [564]:
classifiers = ["bayes", "svm", "sgdc", "randomForest"]
def classifyAndEvaluate(featureSpace, classifier):
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
    clf = None
    if(classifier == "bayes"):
        clf = MultinomialNB()
    elif(classifier == "svm"):
        clf = svm.SVC(gamma="scale")
    elif(classifier == "sgdc"):
        clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
    elif(classifier == "randomForest"):
        clf = RandomForestClassifier(n_estimators=100, random_state=42)
    vectors = featureSpace.values
    annotation = df["majority_sentiment"]
    scores = cross_val_score(clf, vectors, annotation, cv=sss)
    print(scores.mean())
    return scores

In [565]:
def getScores(normalizationApproach, filterApproach, featureMethod, filterFeatureSpace, classifier):
    featureSpace = buildFeatureSpace(normalizationApproach, filterApproach, featureMethod, filterFeatureSpace)
    scores = classifyAndEvaluate(featureSpace, classifier)
    return scores

In [566]:
def compareAndPrint():
    print("\t".join(["Normalization", "Filtering", "Fspace", "Fspace-Filter", "Classifier", "Precision"]))
    for normalization in normalizationMethods:
        for filtering in filterMethods:
            for fspace in featureMethods:
                for fspaceFilter in featureSpaceFilterMethods:
                    for classifier in classifiers:
                        scores = getScores(normalization, filtering, fspace, fspaceFilter, classifier)
                        precision = scores.mean()
                        print("\t".join([normalization, filtering, fspace, fspaceFilter, classifier, str(precision)]))

In [568]:
compareAndPrint()

Normalization	Filtering	Fspace	Fspace-Filter	Classifier	Precision
0.6128851540616246
None	None	tf	None	bayes	0.6128851540616246
0.5747899159663865
None	None	tf	None	svm	0.5747899159663865


C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\s

0.5630252100840336
None	None	tf	None	sgdc	0.5630252100840336
0.5529411764705883
None	None	tf	None	randomForest	0.5529411764705883
0.6408963585434173
None	None	tf	uniqueWords	bayes	0.6408963585434173
0.5820728291316527
None	None	tf	uniqueWords	svm	0.5820728291316527


C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\s

0.565266106442577
None	None	tf	uniqueWords	sgdc	0.565266106442577
0.580392156862745
None	None	tf	uniqueWords	randomForest	0.580392156862745


...

0.5904761904761904
lemmatizationAndLower	punctuationAndStopwords	tf	uniqueWords	svm	0.5904761904761904


C:\Users\Phili\AppData\Local\Temp/ipykernel_22784/591192425.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(dictionary)
C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "
C:\Users\Phili\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:574: 

0.5871148459383754
lemmatizationAndLower	punctuationAndStopwords	tf	uniqueWords	sgdc	0.5871148459383754


C:\Users\Phili\AppData\Local\Temp/ipykernel_22784/591192425.py:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = pd.Series(dictionary)


0.5977591036414566
lemmatizationAndLower	punctuationAndStopwords	tf	uniqueWords	randomForest	0.5977591036414566
